# Let's use emcee to see if we can infer the RV of a star using Adrian's code.

In [9]:
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import aviary as av

from astropy.coordinates.builtin_frames.galactocentric import get_matrix_vectors

Set up the solar position and motion:

In [12]:
sun_xyz = [-8.122, 0, 0] * u.kpc
sun_vxyz = [12.9, 245.6, 7.78] * u.km/u.s

galcen_frame = coord.Galactocentric(galcen_distance=np.abs(sun_xyz[0]),
                                    galcen_v_sun=sun_vxyz,
                                    z_sun=0*u.pc)

# Pre-compute the rotation matrix to go from Galactocentric to ICRS (ra/dec) coordinates
R_gal, _ = get_matrix_vectors(galcen_frame, inverse=True)

#Test the function by transforming a coordinate to galactocentric and back:
c = coord.SkyCoord(ra=61.342*u.deg, 
                   dec=17*u.deg, 
                   distance=3*u.kpc,
                   pm_ra_cosdec=4.2*u.mas/u.yr,
                   pm_dec=-7.2*u.mas/u.yr,
                   radial_velocity=17*u.km/u.s)

test_galcen = c.transform_to(galcen_frame)
print(test_galcen.data.xyz)

test_pm, test_rv = av.get_icrs_from_galactocentric(
    test_galcen.data.xyz,
    test_galcen.velocity.d_xyz, R_gal, sun_xyz, sun_vxyz)
print(test_pm)

assert u.allclose(test_pm[0], c.pm_ra_cosdec)
assert u.allclose(test_pm[1], c.pm_dec)
assert u.allclose(test_rv, c.radial_velocity)

[-10.82072747   0.21518273  -1.29250393] kpc
[ 4.2 -7.2] mas / yr


We want to infer the vz and RV of a star from its proper motion in ra and dec and its observed radial velocity if available (and marginalize over RV if it isn't).

$$p\mathrm{(v_z, RV_{in}|\mu_\alpha, \mu_\delta, RV_{obs})} = p\mathrm{(\mu_\alpha, \mu_\delta, RV_{obs} | v_z, RV_{in})}~p\mathrm{(v_z)}~p\mathrm{(RV_{in})}$$

In [37]:
def lnlike_one_star(pm_obs, pm_err, coords, D, rv):
    c = coord.SkyCoord(ra = coords[0]*u.deg, 
                       dec = coords[1]*u.deg, 
                       distance = D*u.kpc,
                       pm_ra_cosdec = pm_obs[0]*u.mas/u.yr,
                       pm_dec = pm_obs[1]*u.mas/u.yr,
                       radial_velocity = rv*u.km/u.s)
    
    galcen = c.transform_to(galcen_frame)
    
    pm, rv = av.get_icrs_from_galactocentric(xyz, vxyz, R_gal, sun_xyz, sun_vxyz)
    
    return - .5*(pm[0].value - pm_obs[0])**2/pm_err[0]**2 \
            - .5*(pm[1].value - pm_obs[1])**2/pm_err[1]**2

pm_obs = np.array([4.2, -7.2])
pm_err = np.array([.4, .7])
rv_obs = 17
rv_err = 1

print(lnlike(pm_obs, pm_err, rv_obs, rv_err, test_galcen.data.xyz, test_galcen.velocity.d_xyz))
print(lnlike(np.array([5.2, -8.2]), pm_err, rv_obs, rv_err, test_galcen.data.xyz, test_galcen.velocity.d_xyz))

-2.369017719987825e-28
-4.145408163265293
